<a href="https://colab.research.google.com/github/merlin-cba/TP_Reconocimiento_Habla/blob/main/Reconocimiento_de_habla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install espeak


In [ ]:
!pip install pyttsx3


In [ ]:
import pyttsx3

# Instanciar el objeto de la librería pyttsx3
engine = pyttsx3.init()

# Establecer la configuración de la voz
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)  # Escoger una voz en español

# Configurar la salida de audio para que funcione en Colab
engine.setProperty('audioDriver', 'alsa')

# Desactivar la opción de espera de eventos
engine.setProperty('blocking', False)

# Generar voz a partir de un texto
texto = "Hola, soy una voz generada por Python"
engine.say(texto)
engine.runAndWait()
engine.save_to_file('Hola, soy una voz generada por Python', 'output.wav')

In [ ]:
!wget https://github.com/Jakobovski/free-spoken-digit-dataset/archive/refs/heads/master.zip
!unzip master.zip

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Definir función para cargar los datos
def load_data():
  data = []
  labels = []
  for folder in os.listdir('/content/free-spoken-digit-dataset-master/recordings/'):
    for file in os.listdir('/content/free-spoken-digit-dataset-master/recordings/' + folder):
      # Verificar si el elemento es un archivo de audio
      if os.path.isfile('/content/free-spoken-digit-dataset-master/recordings/' + folder + '/' + file):
        # Cargar archivo de audio
        audio_path = '/content/free-spoken-digit-dataset-master/recordings/' + folder + '/' + file
  signal, sr = librosa.load(audio_path)
  # Convertir señal de audio en espectrograma
  spectrogram = librosa.feature.melspectrogram(signal, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
  spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
  # Añadir espectrograma y etiqueta a los datos
  data.append(spectrogram)
  labels.append(int(folder))
  # Convertir listas a numpy arrays
  data = np.array(data)
  labels = np.array(labels)
  return data, labels

# Cargar datos
data, labels = load_data()

# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Crear modelo de red neuronal utilizando TensorFlow
model = tf.keras.models.Sequential([
tf.keras.layers.Input(shape=(128, 32, 1)),
tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(256, activation='relu'),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(10, activation='softmax')
])

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_data, train_labels, batch_size=32, epochs=10, validation_data=(test_data, test_labels))

# Evaluar el modelo
loss, accuracy = model.evaluate(test_data, test_labels)
print(f'Loss: {loss:.3f}, Accuracy: {accuracy:.3f}')